<a href="https://colab.research.google.com/github/sohaamir/aamir_vs_chris/blob/main/aamir_vs_chris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Who has the bigger brain, me or Chris Gorgolewski? A gentle introduction to nipype**







## What is Google Colab?
**Google Colaboratory** is is a free cloud-based platform provided by Google that offers a Jupyter notebook environment for writing and executing Python code. It is primarily used for data analysis tasks but can also be used for general-purpose Python programming. It has many benefits when working with data (including neuroimaging data) such as:

1.   Free access
2.   Cloud-based hosting (available everywhere)
3.   GPU/TPU Support (lots of processing power)
4.   External Data Access (import data GitHub, Google Drive, local machine)

It is an interactive environment similar to Anaconda, but with certain advantages (like those mentioned above). Similarly, Colab allows for users to run code in small chunks called 'cells', displaying any output such as images directly within the notebook as well. The programming language used by these notebooks is `Python`, which it organises in the form of `Jupyter Notebooks`.

## What can we do in Google Colab?

We can do a whole bunch of things...

### We can use it for data visualization and plotting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 10, 100)
y = np.sin(x)

plt.plot(x, y)
plt.title("Sine Wave")
plt.xlabel("X")
plt.ylabel("sin(X)")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def mandelbrot(c, max_iter):
    z = 0
    n = 0
    while abs(z) <= 2 and n < max_iter:
        z = z*z + c
        n += 1
    return n

def mandelbrot_image(xmin, xmax, ymin, ymax, width=10, height=10, max_iter=256):
    # Create a width x height grid of complex numbers
    x = np.linspace(xmin, xmax, width * 100)
    y = np.linspace(ymin, ymax, height * 100)
    X, Y = np.meshgrid(x, y)
    C = X + 1j * Y

    # Compute Mandelbrot set
    img = np.zeros(C.shape, dtype=int)
    for i in range(width * 100):
        for j in range(height * 100):
            img[j, i] = mandelbrot(C[j, i], max_iter)

    # Plotting
    plt.figure(figsize=(width, height))
    plt.imshow(img, extent=(xmin, xmax, ymin, ymax), cmap='hot')
    plt.colorbar()
    plt.title("Mandelbrot Set")
    plt.show()

# Parameters defining the extent of the region in the complex plane we're going to plot
xmin, xmax, ymin, ymax = -2.0, 0.5, -1.25, 1.25
width, height = 10, 10
max_iter = 256

mandelbrot_image(xmin, xmax, ymin, ymax, width, height, max_iter)

### Watch YouTube

In [ ]:
from IPython.display import YouTubeVideo

# Embedding the YouTube video
YouTubeVideo('GtL1huin9EE', width=800, height=450)

## Learning about neuroimaging

## What is nipype?
nipype, to quote the website aims to "provide an environment that encourages interactive exploration of algorithms from different packages (e.g., ANTS, SPM, **FSL**, FreeSurfer, Camino, MRtrix, MNE, AFNI, Slicer, DIPY), eases the design of workflows within and between packages, and reduces the learning curve necessary to use different packages."

To quote [Nick Hedger](https://github.com/N-HEDGER/):

> **The simplest way of thinking about nipype is as an interface (or set of interfaces) that allow all of our favourite neuroimaging packages to talk to one another and work effectively together.**


In [ ]:
from IPython.display import Image
from IPython.display import display, HTML

Image(url='https://www.researchgate.net/publication/349017135/figure/fig1/AS:987364692328449@1612417495314/Neuroimaging-analysis-pipeline-A-An-automated-NIPYPE-pipeline-extracted-structural-and.jpg')

In [ ]:
# Step 1: Install necessary packages
!pip install nibabel nipype matplotlib
!pip install imageio

# Step 2: Download the MRI scan files from GitHub
!wget https://github.com/sohaamir/aamir_vs_chris/raw/main/sub-CG_T1w.nii -O sub-CG_T1w.nii
!wget https://github.com/sohaamir/aamir_vs_chris/raw/main/sub-0002_ses-1_acq-mprage_rec-NORM_T1w.nii -O sub-0002_ses-1_acq-mprage_rec-NORM_T1w.nii

import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from nipype.interfaces import fsl
import imageio

In [ ]:
# Function to load and visualize the MRI scans
def visualize_mri(path, slice_index):
    # Load the image
    img = nib.load(path)
    data = img.get_fdata()

    # Plot the image
    plt.imshow(data[:, :, slice_index], cmap='gray')
    plt.show()

    return img

In [ ]:
# Visualize a slice of the first MRI
img1 = visualize_mri('sub-CG_T1w.nii', slice_index=100)

# Visualize a slice of the second MRI
img2 = visualize_mri('sub-0002_ses-1_acq-mprage_rec-NORM_T1w.nii', slice_index=100)

In [12]:
# Function to normalize and create a GIF from a 3D MRI scan in the sagittal plane
def create_gif_from_mri_normalized(path, gif_name):
    # Load the image and get the data
    img = nib.load(path)
    data = img.get_fdata()

    # Normalize the data for better visualization
    # Clip the top and bottom 1% of pixel intensities
    p2, p98 = np.percentile(data, (2, 98))
    data = np.clip(data, p2, p98)
    data = (data - np.min(data)) / (np.max(data) - np.min(data))

    # Prepare to capture the slices
    slices = []
    # Sagittal slices are along the x-axis, hence data[x, :, :]
    for i in range(data.shape[0]):
        slice = data[i, :, :]
        slice = np.rot90(slice)  # Rotate or flip the slice if necessary
        slices.append((slice * 255).astype(np.uint8))  # Convert to uint8 for GIF

    # Create a GIF
    imageio.mimsave(gif_name, slices, duration=0.1)  # duration controls the speed of the GIF

In [ ]:
from IPython.display import Image, display

# Create and display the GIF for the first MRI
create_gif_from_mri_normalized('sub-CG_T1w.nii', 'brain1_normalized.gif')
display(Image(filename='brain1_normalized.gif'))

In [ ]:
# Create and display the GIF for the second MRI
create_gif_from_mri_normalized('sub-0002_ses-1_acq-mprage_rec-NORM_T1w.nii', 'brain2_normalized.gif')
display(Image(filename='brain2_normalized.gif'))

In [ ]:
# Step 5: Calculate the size of the brains
# Here we'll just count the non-zero voxels as a proxy for brain size
# This is a naive approach; in practice you would want a more sophisticated method
size_img1 = np.count_nonzero(img1.get_fdata())
size_img2 = np.count_nonzero(img2.get_fdata())

print(f"Size of brain in the first image: {size_img1} voxels")
print(f"Size of brain in the second image: {size_img2} voxels")

# Compare sizes
if size_img1 > size_img2:
    print("The first brain is larger.")
elif size_img1 < size_img2:
    print("The second brain is larger.")
else:
    print("The brains are the same size.")